### Libraries as necessary

In [13]:
import polars as pl

from glob import glob # reading multiple files in the same directory
from datetime import datetime

from tqdm import tqdm # progress bar
import shutup # it just makes me feel better
shutup.please()

In [14]:
def find_files(paths, years=False, extension="", lookback=0):
    '''
    Find files that match the specified criteria.

    Parameters
    ----------
    paths : str or list of str
        Folder(s) to look for the data files. It MUST be a raw string (r"").
    years : str or list of str, oprional
        Year of the datafile to look for.
        If multiple years are provided in a list, it is considered a range.
        If not provided, it looks for all files. then lookback is ignored.
    extension : str or list of str, optional
        File types to look for.
        The default is any extension.
    lookback : int, optional
        The number of years to look back. For orders, this should be 1 since the medication can be ordered first and then billed later.
        If multiple years are provided in a list, it looks back from the minimum year.
        The default is 0.

    Returns
    -------
    files : list
        List of files that match the criteria.
    '''

    if not isinstance(paths, list):
        paths = [paths]

    if not isinstance(extension, list):
        extension = [extension]

    if years:
        years = list(map(int, years))
        year_list = list(range(min(years) - lookback, max(years) + 1))

    paths = ([path + "/*" + ext for path in paths for ext in extension])
    files  = [glob(path) for path in paths]
    files = [file for filelist in files for file in filelist]

    files = [file for file in files if any(str(year) in file.split('\\')[-1] for year in year_list)]
        
    return files

### ICD-10 criteria

ICD-10 codes used to identify are I10, I11, I12, I13 and I15. 

``str(x).startswith(tuple(icd10))`` is used because ICD-10 code is Alpha character followed by multiple numerical characters (A12344). So, ``startswith`` catches all I10, I100, I101, I1010 and so on.

This is the required input format. <br /> 

| Primary Key | Date | ICD code |
|-|-|-|
| A | 2023-01-01 | (not relevant) |
| A | 2023-01-10 | (relevant) |
| A | 2023-01-15 | (relevant) |
| A | 2023-02-01 | (relevant) |

The first date of diagnosis for each of these codes is considered as date of HT diagnosis, so only the earliest date is kept. <br /> 

| Primary Key | Date | ICD code |
|-|-|-|
| A | 2023-01-10 | (relevant) |

In [15]:
start  = datetime.strptime("20230701", "%Y%m%d")

In [17]:
# Diagnosis

icd10 = ["I10", "I11", "I12", "I13", "I15"]

paths = "folders including diagnosis data"

files = find_files(paths, years="2023")
read = []
for file in tqdm(files):
    temp = pl.scan_parquet(file)
    temp = (temp.select(["ENC_HN", "D001KEY", "D035KEY"]).with_columns(pl.col("D001KEY").cast(pl.Date))
            .filter(pl.any_horizontal(pl.col("D035KEY").str.starts_with(code) for code in icd10))
            .filter(pl.col("D001KEY")>=start).groupby(["ENC_HN", "D001KEY"]).agg(pl.col("D035KEY").unique())
            .sort(["ENC_HN", "D001KEY"]).unique(subset=["ENC_HN"]))
    read.append(temp)

dx = pl.concat(read).collect()

100%|██████████| 14/14 [00:00<00:00, 71.37it/s]


### Medication criteria

The same goes for medication. We identified the medications prescribed in Ramathibodi Hospital and filtered for relevant drug classes. The mapping table was prepared by Dr Kunlawat Thadanipon and Dr Thosaphol Limpijankit.

In [21]:
"Medication"

key = pl.read_excel(r"map_list\cohort_anti_HT.xlsx", sheet_name="selected drug classes")
key.head()

ramadrugcode,drugname,combination,combinationsplit,atc3,atc3des,atc5,atc5des,subclass,drugclass,starting
str,str,str,str,str,str,str,str,str,str,str
"""xxxxxx-""","""Accupril 20 mg…","""Quinapril""","""Quinapril""","""C09A""","""ACE INHIBITORS…","""C09AA06""","""Quinapril""","""ACEI""","""ACEI""",null
"""xxxxxx-""","""Accupril 10 mg…","""Quinapril""","""Quinapril""","""C09A""","""ACE INHIBITORS…","""C09AA06""","""Quinapril""","""ACEI""","""ACEI""",null
"""xxxxxx-""","""Accupril 40 mg…","""Quinapril""","""Quinapril""","""C09A""","""ACE INHIBITORS…","""C09AA06""","""Quinapril""","""ACEI""","""ACEI""",null
"""xxxxxx-""","""Accupril 5 mg""","""Quinapril""","""Quinapril""","""C09A""","""ACE INHIBITORS…","""C09AA06""","""Quinapril""","""ACEI""","""ACEI""",null
"""xxxxxx-""","""Accuretic 10/1…","""Quinapril+Hydr…","""Quinapril""","""C09B""","""ACE INHIBITORS…","""C09BA06""","""Quinapril and …","""ACEI""","""ACEI""",null


This is the required input format. <br /> 

| Primary Key | Date | Drugcode |
|-|-|-|
| A | 2023-01-01 | (not relevant) |
| A | 2023-01-10 | (relevant) |
| A | 2023-01-10 | (relevant) |
| A | 2023-02-01 | (relevant) |

Essentially the goal is to identify the transactions where relevant Anti Hypertensive medication was given, and keep the earliest date. Please note that multiple relevant medications can be prescribed on the same day. <br /> 

| Primary Key | Date | Drugcode |
|-|-|-|
| A | 2023-01-10 | (relevant) |
| A | 2023-01-10 | (relevant) |


In [24]:
paths = "folders including medication data"

files = find_files(paths, years="2023")

read = []
for file in tqdm(files):
    temp = pl.scan_parquet(file)
    temp = (temp.with_columns(pl.col("BillDate").fill_null(pl.col("OrderDate")))
            .select(["ENC_HN", "BillDate", "Drugcode"]).with_columns(pl.col("BillDate").cast(pl.Date))
            .filter(pl.col("Drugcode").is_in(key["ramadrugcode"]))
            .filter(pl.col("BillDate")>=start).groupby(["ENC_HN", "BillDate"]).agg(pl.col("Drugcode").unique())
            .sort(["ENC_HN", "BillDate"]).unique(subset=["ENC_HN"]))
    read.append(temp)

med = pl.concat(read).collect()

100%|██████████| 1/1 [00:00<00:00, 83.28it/s]


### New cases of Hypertension

This step is not necessary for the first time. For me, I am updating my cohort so subjects already in the cohort are removed.

In [26]:
# because it is an update, I remove old cases.

old_hn = pl.read_csv(r"path\2010-202306\hthn_2010_202306.csv")

new_dx = dx.filter(~pl.col("ENC_HN").is_in(old_hn["ENC_HN"]))
new_med = med.filter(~pl.col("ENC_HN").is_in(old_hn["ENC_HN"]))

# these numbers are up to changes since new_dx can be med_only if prescription is observed earlier than diagnosis

### Merging for tentative number of cases

This is diagnosis data. <br /> 
| Primary Key | Date | ICD |
|-|-|-|
| A | 2023-01-09 | (icd code) |
| C | 2023-02-13 | (icd code) |

This is medication data. <br /> 
| Primary Key | Date | MED |
|-|-|-|
| A | 2023-01-10 | (med code) |
| B | 2023-01-13 | (med code) |
| C | 2023-02-13 | (med code) |

This is merging for tentative number of cases. While the patient might be in both ICD and Medication groups, we identify by the first date of diagnosis. <br /> 

| Primary Key | ICD_Date | Med_Date | First_Date | flag |
|-|-|-|-|-|
| A | 2023-01-09 | 2023-01-10 | 2023-01-09 | icd |
| B | | 2023-01-13 | 2023-01-13 | med |
| C | 2023-02-13 | 2023-02-13 | 2023-02-13 | both |

In these subjects, subject A is considered ICD only, subject B as Anti-H only and C as both ICD-10 and Anti-H group.



In [27]:
new_dx.columns = ["ENC_HN", "ICD_DATE", "ICD"]
new_med.columns = ["ENC_HN", "MED_DATE", "MED"]

data = (new_dx.join(new_med, on = ["ENC_HN"], how="outer_coalesce")
        .with_columns(pl.min_horizontal(["ICD_DATE", "MED_DATE"]).alias("D001KEY"))
        .with_columns(pl.when((pl.col("D001KEY")==pl.col("ICD_DATE")) & (pl.col("D001KEY")==pl.col("MED_DATE"))).then(pl.lit("both"))
                    .otherwise(pl.when((pl.col("D001KEY")==pl.col("ICD_DATE"))).then(pl.lit("icd"))
                                .otherwise(pl.lit("med"))).alias("flag")))

print("No of possible HT subjects are {:,}.".format(data["ENC_HN"].n_unique()))

for flag in ["icd", "both", "med"]:
    print("For {} group, the number of subjects is {:,}.".format(flag, len(data.filter(pl.col("flag")==flag))))


# data.write_parquet(r"path/new_case/merged_hn_inferred.parquet.gzip")

No of possible HT subjects are 6,800.
For icd group, the number of subjects is 2,227.
For both group, the number of subjects is 758.
For med group, the number of subjects is 3,815.


### Filtering the medication only cases

Since medications are identified using ramadrugcodes, they are transformed into generic drug classes using the previously shown mapping table.

``ast.literal_eval`` is required if the data file is imported again since Pandas does not import ``dtype`` as ``list``.

In [28]:
key = pl.read_excel(r"path\map_list\cohort_anti_HT.xlsx", sheet_name="selected drug classes")
key = dict(key.select(["ramadrugcode", "subclass"]).iter_rows())

med_only = data.filter(pl.col("flag")=="med") # med only group
med_only = (med_only.join(med_only.explode("MED")
                          .with_columns(pl.col("MED").replace(key).alias("drugclass"))
                          .group_by("ENC_HN").agg(pl.col("drugclass").unique()), 
                          on="ENC_HN"))

print("\nMapping ramadrugcodes such as {} to generic drugclass such as {}".format(med_only["MED"][0][0], med_only["drugclass"][0][0]))
med_only[["MED", "drugclass"]].head()


Mapping ramadrugcodes such as CRDP1I- to generic drugclass such as CCB_DHP


MED,drugclass
list[str],list[str]
"[""CRDP1I-""]","[""CCB_DHP""]"
"[""PPNL1T-""]","[""beta_blocker""]"
"[""MADI1T-"", ""HYDH-T-""]","[""hydralazine"", ""CCB_DHP""]"
"[""DOXA1T-""]","[""alpha_blocker""]"
"[""PPNL1T-""]","[""beta_blocker""]"


As of 2024 January, we check for gender specific medication usages, such as K-sparing Diuretics for Acnes in Female patients. Gender is added.

In [29]:
demo = pl.scan_parquet(r"demographic_data.parquet.gzip")

med_only = med_only.join(demo.select(["ENC_HN", "H2L1KEY"]).unique("ENC_HN")
                         .rename({"H2L1KEY": "Gender"})
                         .with_columns(pl.col("Gender").replace({"M": "Male", "F": "Female"}))
                         .collect(), on="ENC_HN")

Other indications for specific drug classes were prepared by Dr Kunlawat Thadanipon and Dr Thosaphol Limpijankit. The function ``check_history`` look for other indications in patient's history using ICD-10 codes.

The input is list of medications and the date of prescription to check. <br /> 

| Primary Key | Date | drugclass |
|-|-|-|
| A | 2023-01-10 | [drug A, drug B] |

If they have corresponding history, the output is tuple of indication and medication. <br /> 

| Primary Key | Date | drugclass | Check History|
|-|-|-|-|
| A | 2023-01-10 | [drug A, drug B] | [drug A, (other indication on 2022-10-04, drug B)] |

If the patient have at least one Anti-H that does not correspond with other indication, we include them.

In [32]:
# Other indication

indi = pl.read_excel(r"path\map_list\medication_map_2401.xlsx", sheet_name="other indications")

indi = (indi.select(pl.exclude("ICD-10")).join((
        indi.with_columns(pl.col("ICD-10").str.replace_all(r"\.", "").str.split(","))
        .explode("ICD-10").with_columns(pl.col("ICD-10").str.strip().str.ljust(4, "0"))
        .groupby("subclass", "Other Indications").agg(pl.col("ICD-10").unique())), on=["subclass", "Other Indications"]))

paths = "folders including diagnosis data"

files = find_files(paths, years=["2010", "2023"])

read = []
for file in files:
    temp = pl.scan_parquet(files)
    temp = (temp.select(["ENC_HN", "D001KEY", "D035KEY"])
            .filter(pl.col("ENC_HN").is_in(med_only["ENC_HN"]))
            .filter(pl.col("D035KEY").is_in(indi["ICD-10"].explode().unique()))
            .with_columns(pl.col("D001KEY").cast(pl.Date)))
    read.append(temp)

dx = pl.concat(read).collect()

In [35]:
history = []
for subclass in indi["subclass"].unique():
    subject_check = med_only.explode("drugclass").filter(pl.col("drugclass")==subclass)
    check = (indi.filter(pl.col("subclass")==subclass)
            ["Other Indications"].unique())
    for val in check:
        indi_check = indi.filter(pl.col("Other Indications")==val)
        if any(indi_check["Gender"]=="Female"):
            subject_check = subject_check.filter(pl.col("Gender")=="Female")
        subject = (dx.filter(pl.col("ENC_HN").is_in(subject_check["ENC_HN"]))
                   .filter(pl.col("D035KEY").is_in(indi_check["ICD-10"].explode().unique()))
                   .join(subject_check.select(["ENC_HN", "MED_DATE"]), on="ENC_HN", how="left")
                   .filter(pl.col("D001KEY")<=pl.col("MED_DATE")).sort(["ENC_HN", "D001KEY"]).unique("ENC_HN")
                   .with_columns(check_history = val + " on " + pl.col("D001KEY").cast(pl.String)))
        history.append(subject)

history = (pl.concat(history)
           .group_by("ENC_HN").agg(pl.col("check_history").unique())
           .select(["ENC_HN", "check_history"]))

to_drop = med_only.join(history, on="ENC_HN", how="inner")

to_drop.write_excel(r"path/new_case/med_only_other_indication_all_des2.xlsx")

pl.concat([pl.DataFrame({"ENC_HN": ["A", "B", "C", "D", "E"]}), to_drop[["D001KEY", "ICD_DATE", "ICD", "MED_DATE", "drugclass", "flag", "check_history"]].head(5)], how="horizontal")

ENC_HN,D001KEY,ICD_DATE,ICD,MED_DATE,drugclass,flag,check_history
str,date,date,list[str],date,list[str],str,list[str]
"""A""",2023-12-22,null,null,2023-12-22,"[""beta_blocker""]","""med""","[""Hyperthyroidism on 2023-12-22""]"
"""B""",2023-11-13,null,null,2023-11-13,"[""beta_blocker""]","""med""","[""Atrial fibrillation on 2022-08-09""]"
"""C""",2023-11-27,null,null,2023-11-27,"[""beta_blocker""]","""med""","[""Hyperthyroidism on 2015-07-06""]"
"""D""",2023-08-10,null,null,2023-08-10,"[""beta_blocker""]","""med""","[""Hyperthyroidism on 2015-05-01""]"
"""E""",2023-10-18,2023-10-22,"[""I10""]",2023-10-18,"[""CCB_nonDHP""]","""med""","[""Arrhythmias on 2023-10-17""]"


### Final Merge

Diagnosis of HT is inferred from medication, after we have dropped those subjects.

In [36]:
final_data = data.filter(~pl.col("ENC_HN").is_in(to_drop["ENC_HN"]))

print("No of possible HT subjects are {:,}.".format(final_data["ENC_HN"].n_unique()))

for flag in ["icd", "both", "med"]:
    print("For {} group, the number of subjects is {:,}.".format(flag, len(final_data.filter(pl.col("flag")==flag))))

pl.concat([pl.DataFrame({"ENC_HN": ["A", "B", "C", "D", "E"]}), final_data[["D001KEY", "ICD_DATE", "ICD", "MED_DATE", "MED", "flag"]].head(5)], how="horizontal")

No of possible HT subjects are 6,657.
For icd group, the number of subjects is 2,227.
For both group, the number of subjects is 758.
For med group, the number of subjects is 3,672.


ENC_HN,D001KEY,ICD_DATE,ICD,MED_DATE,MED,flag
str,date,date,list[str],date,list[str],str
"""A""",2023-09-07,null,null,2023-09-07,"[""DOXA1T-""]","""med"""
"""B""",2023-12-25,2023-12-27,"[""I10""]",2023-12-25,"[""IRBW1T-"", ""DILZ1T-"", ""HCTZ1T-""]","""med"""
"""C""",2023-09-14,2023-09-14,"[""I129""]",2023-11-30,"[""AMLP1T-"", ""BIBI1T-""]","""icd"""
"""D""",2023-11-25,2023-12-03,"[""I10""]",2023-11-25,"[""MADI1T-"", ""HYDH-T-""]","""med"""
"""E""",2023-12-18,null,null,2023-12-18,"[""LOSL-T-""]","""med"""
